# Необходимые функции

In [1]:
from random import randint
from sympy import randprime 
import time

def st(n):
    s = 0
    t = n
    while t % 2 == 0:
        s += 1
        t = t // 2
    return s, t

def Jacobi(a,n):
    if n < 0 or not n % 2:
        raise ValueError("n should be an odd positive integer") 
    j = 1
    if n == 1:
        return j
    if a<0:
        a = -a
        if n%4 == 3:
            j = -j
    while n>1:
        if a == 0:
            return 0
        s,t = st(a)
        if (s%2 == 1) & (n%8 in [3, 5]):
            j = -j
        if 3 == n%4 == t%4:
            j = -j
        a = n%t
        n = t
    return j

def Shanks(a, p):
    if a % p == 0:
        return 0
    if not Jacobi(a, p) == 1: 
        return "no solution"
    s,t = st(p-1)             
    n = randint(2,p-2) 
    while Jacobi(n,p) == 1:   
        n = randint(2,p-2)
    b = pow(n,t,p)            
    r = pow(a,(t+1)//2,p)     
    d = 0
    f = pow(a,t,p)            
    b2 = b
    for i in range(1,s):      
        b2 = b2*b2 % p        
        if not pow(f,2**(s-1-i), p) == 1:  
            d += 2**i                   
            f = f*b2 % p                 
    return r*pow(b,d//2,p) % p          

In [2]:
def D(a,b):
    return 27*b**2 + 4*a**3

# Аффинные координаты

## Реализация

In [3]:
class Point:
    def __init__(self,x,y, curve):
        self.curve = curve
        x %= self.curve.p
        y %= self.curve.p
        if not self.curve.isPoint(x,y):
            print("Ошибка! Точка не лежит на кривой!")
        else:
            self.x = x 
            self.y = y 
            
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.x == other.x) and (self.y == other.y) and (self.curve == other.curve)
        else:
            return False
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    def __neg__(self):
        return Point(self.x, -self.y, self.curve)
    
    def doubling(self):
        y1 = self.y
        x1 = self.x
        if y1 == 0:
            return infPoint(self.curve)
        else:
            k = (3*(x1*x1 % self.curve.p) + self.curve.a)*pow((2*y1),-1, self.curve.p) % p  
            x3 = k*k % self.curve.p - 2*x1            
            y3 = k*(x1-x3) - y1        
            return Point(x3,y3, self.curve)
        
    def __add__(self, P):
        if isinstance(P, infPoint):
            return self
        y1 = self.y
        x1 = self.x
        y2 = P.y
        x2 = P.x
        if x1 == x2:
            if y1 == y2 == 0:
                return infPoint(self.curve)
            elif y1 != y2:
                return infPoint(self.curve)
            else:
                k = (3*(x1*x1 % self.curve.p) + self.curve.a)*pow((2*y1),-1, self.curve.p) % p  
                x3 = k*k % self.curve.p - 2*x1            
                y3 = k*(x1-x3) - y1        
                return Point(x3,y3, self.curve)
        k = (y2-y1)*pow((x2 - x1), -1, self.curve.p) % self.curve.p       
        x3 = k*k % self.curve.p - x1- x2                        
        y3 = k*(x1-x3) - y1                     
        return Point(x3,y3, self.curve)
        
    def __sub__(self,P):
        return self + (-P)
    
    def __mul__(self, m):
        if isinstance(m, int):
            if m < 0:
                m = -m
                self = -self
            m = bin(m)[2:]
            result = infPoint(self.curve)
            Q = self
            for i in m[::-1]:
                if i == "1":
                    result = result + Q
                Q += Q
            return result
        else:
            return f"{m} должно быть целым числом"
        
    def __rmul__(self,m):
        return self*m
    
    def toProjective(self):
        return ProjectivePoint(self.x,self.y,1,self.curve.toProjective())
    
    def toJacobian(self):
        return JacobianPoint(self.x,self.y,1,self.curve.toJacobian())
    
    def toChudnovskyJacobian(self):
        return ChudnovskyJacobianPoint(self.x,self.y,1,1,1,self.curve.toChudnovskyJacobian())

In [4]:
class infPoint():
    def __init__(self, curve):
        self.curve = curve
    def __neg__(self):
        return self  
    def __add__(self, P):
        return P
    def __sub__(self,P):
        return self + (-P)
    def __repr__(self):
        return "O"
    def __mul__(self, m):
        return self
    def __rmul__(self, m):
        return self
    def toProjective(self):
        return ProjectivePoint(0,1,0,self.curve.toProjective())
    def toJacobian(self):
        return JacobianPoint(1,1,0,self.curve.toJacobian())
    def toChudnovskyJacobian(self):
        return ChudnovskyJacobianPoint(1,1,0,0,0,self.curve.toChudnovskyJacobian())

In [5]:
class Curve:
    def __init__(self,a,b,p):
        if D(a,b) % p: 
            self.a = a % p
            self.b = b % p
            self.p = p
        else:
            print("Ошибка! Дискриминант кривой нулевой!")
            
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.a == other.a) and (self.b == other.b) and (self.p == other.p)
        else:
            return False
    
    def findPointsShanks(self):
        points = []
        for x in range(self.p):
            a = (pow(x,3,self.p) + self.a*x + self.b) % self.p
            y = Shanks(a, self.p)
            if isinstance(y, int):
                points.append(Point(x,y,self))
                if y != ((self.p - y) % self.p):
                    points.append(Point(x,-y, self))
        points.append(infPoint(self))
        return points
    
    def findRandomPoint(self):
        while True:
            x = randint(0, self.p - 1)
            a = (pow(x,3,self.p) + self.a*x + self.b) % self.p
            y = Shanks(a, self.p)
            if isinstance(y, int):
                return Point(x,y, self)
    
    def isPoint(self,x,y):
        return not ((pow(y,2,self.p) - (pow(x,3,self.p) + self.a*x + self.b)) % self.p)
        
    def toProjective(self):
        return ProjectiveCurve(self.a,self.b,self.p)
    
    def toJacobian(self):
        return JacobianCurve(self.a,self.b,self.p)
    
    def toChudnovskyJacobian(self):
        return ChudnovskyJacobianCurve(self.a,self.b,self.p)

## Асcоциативность, коммутативность

In [6]:
a = randint(-1000,1000)
b = randint(-1000,1000)
p = randprime(1<<16, 1<<17)
ec1 = Curve(a,b,p)
p1 = ec1.findRandomPoint()
p2 = ec1.findRandomPoint()
p3 = ec1.findRandomPoint()
print("Ассоциативность: ", (p1 + p2) + p3 == p1 + (p2 + p3))
print("Коммутативность: ", p1 + p2 == p2 + p1)

Ассоциативность:  True
Коммутативность:  True


# Проективные координаты

## Реализация

In [7]:
class ProjectiveCurve:
    def __init__(self, a, b, p):
        if D(a,b) % p: 
            self.a = a % p
            self.b = b % p
            self.p = p
        else:
            print("Ошибка! Дискриминант кривой нулевой!")
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.a == other.a) and (self.b == other.b) and (self.p == other.p)
        else:
            return False
    
    def findRandomPoint(self):
        while True:
            x = randint(0, self.p - 1)
            y = randint(0, self.p - 1)
            z = randint(1, self.p - 1)
            if not ((pow(y,2,self.p)*z - (pow(x,3,self.p) + self.a*x*pow(z,2,self.p) + self.b*pow(z,3,self.p))) % self.p):
                return ProjectivePoint(x,y,z,self)
            
    def findPoints(self):
        points = []
        for x in range(self.p):
            for y in range(self.p):
                for z in range(1, self.p):
                    if not((pow(y,2,self.p)*z - (pow(x,3,self.p) + self.a*x*pow(z,2,self.p) + self.b*pow(z,3,self.p))) % self.p):
                        if ProjectivePoint(x,y,z, self) not in points:
                            points.append(ProjectivePoint(x,y,z, self))
        points.append(ProjectivePoint(0,1,0,self))
        return points

    def isPoint(self,x,y,z):
        return not ((pow(y,2,self.p)*z - (pow(x,3,self.p) + self.a*x*pow(z,2,self.p) + self.b*pow(z,3,self.p))) % self.p)
        
    def toAffine(self):
        return Curve(self.a,self.b,self.p)

In [8]:
class ProjectivePoint:
    def __init__(self,x,y,z, curve, secure = True):
        self.curve = curve
        x %= self.curve.p
        y %= self.curve.p
        z %= self.curve.p
        if not secure:
            if not self.curve.isPoint(x,y,z):
                print(x,y,z)
                print("Ошибка! Точка не лежит на кривой!")
                secure = False
            else:
                secure = True
        if secure:
            self.x = x 
            self.y = y 
            self.z = z 
            
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.curve == other.curve) and (not ((self.x*other.y - self.y*other.x)) % self.curve.p) and (not ((self.y*other.z-self.z*other.y))% self.curve.p)
        else:
            return False
        
    def __repr__(self):
        return f"({self.x}, {self.y}, {self.z})"
    
    def __neg__(self):
        return ProjectivePoint(self.x, -self.y, self.z, self.curve)
    
    def doubling(self):
        y1 = self.y
        x1 = self.x
        z1 = self.z
        if y1 == 0:
                return ProjectivePoint(0,1,0,self.curve)
        w = (self.curve.a*z1*z1 % self.curve.p + 3*x1*x1 % self.curve.p) % self.curve.p             
        s = y1*z1 % self.curve.p   
        s2 = s*s % self.curve.p
        s3 = s2*s % self.curve.p
        B = x1*y1*s % self.curve.p     
        h = w*w % self.curve.p - 8*B
        x3 = 2*h*s 
        y3 = w*(4*B - h) - 8*(y1*y1 % self.curve.p)*s2
        z3 = 8*s3
        return ProjectivePoint(x3,y3,z3, self.curve)
    
    def __add__(self, P):
        y1 = self.y
        x1 = self.x
        z1 = self.z
        y2 = P.y
        x2 = P.x
        z2 = P.z
        if z1 == 0:
            return P
        elif z2 == 0:
            return self
        if P == self:
            if y1 == 0:
                return ProjectivePoint(0,1,0,self.curve)
            w = (self.curve.a*z1*z1 % self.curve.p + 3*x1*x1 % self.curve.p) % self.curve.p             
            s = y1*z1 % self.curve.p   
            s2 = s*s % self.curve.p
            s3 = s2*s % self.curve.p
            B = x1*y1*s % self.curve.p     
            h = w*w % self.curve.p - 8*B
            x3 = 2*h*s 
            y3 = w*(4*B - h) - 8*(y1*y1 % self.curve.p)*s2
            z3 = 8*s3
            return ProjectivePoint(x3,y3,z3, self.curve)
        
        v = x2*z1 - x1*z2 % self.curve.p
        v2 = v*v % self.curve.p
        v3 = v2*v % self.curve.p
        u = y2*z1 - y1*z2 % self.curve.p
        u2 = u*u % self.curve.p
        A = u2*z1*z2 - v3 - 2*v2*x1*z2 % self.curve.p     
        x3 = v*A
        y3 = u*(v2*x1*z2 - A) - v3*y1*z2
        z3 = v3*z1*z2
        return ProjectivePoint(x3,y3,z3, self.curve)
        
    def __sub__(self,P):
        return self + (-P)
    
    def toAffine(self):
        curve = self.curve.toAffine()
        if self.z == 0:
            return infPoint(curve)
        return Point(self.x*pow(self.z, -1, self.curve.p), self.y*pow(self.z, -1, self.curve.p), curve)
    
    def __mul__(self, m):
        if isinstance(m, int):
            if m < 0:
                m = -m
                self = -self
                
            m = bin(m)[2:]
            result = ProjectivePoint(0,1,0,self.curve)
            Q = self
            for i in m[::-1]:
                if i == "1":
                    result = result + Q
                Q += Q
            return result
        else:
            return f"{m} должно быть целым числом"
        
    def __rmul__(self,m):
        return self*m

## Асcоциативность, коммутативность

In [9]:
a = randint(-1000,100)
b = randint(-100,1000)
p = randprime(1<<12, 1<<13)
pc1 = ProjectiveCurve(a,b,p)
pp1 = pc1.findRandomPoint()
pp2 = pc1.findRandomPoint()
pp3 = pc1.findRandomPoint()
print("Ассоциативность: ", (pp1 + pp2) + pp3 == pp1 + (pp2 + pp3))
print("Коммутативность: ", pp1 + pp2 == pp2 + pp1)

Ассоциативность:  True
Коммутативность:  True


## Другие тесты

In [10]:
a = randint(-1000,100)
b = randint(-100,1000)
p = randprime(1<<12, 1<<13)
pc1 = ProjectiveCurve(a,b,p)
ec1 = pc1.toAffine()
pp1 = pc1.findRandomPoint()
p1 = pp1.toAffine()

print((p1*b).toProjective() == pp1*b)
(pp1*a).toAffine() == p1*a # методы перевода

True


True

In [11]:
a = randint(-100,100)
b = randint(-100,100)
p = randprime(1<<5, 1<<6)
pc1 = ProjectiveCurve(a,b,p)
ec1 = pc1.toAffine()

In [12]:
pc1 = ProjectiveCurve(2,5,17)
ppp1 = ProjectivePoint(1,3,7,pc1)
ppp2 = ProjectivePoint(2,6,14,pc1)
ppp1 == ppp2 # эквивалентность точек

True

# Координаты Якоби

## Реализация

In [13]:
class JacobianCurve:
    def __init__(self, a, b, p):
        if D(a,b) % p: 
            self.a = a % p
            self.b = b % p
            self.p = p
        else:
            print("Ошибка! Дискриминант кривой нулевой!")
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.a == other.a) and (self.b == other.b) and (self.p == other.p)
        else:
            return False
    
    def findRandomPoint(self):
        while True:
            x = randint(0, self.p - 1)
            y = randint(0, self.p - 1)
            z = randint(1, self.p - 1)
            if not ((pow(y,2,self.p) - (pow(x,3,self.p) + self.a*x*pow(z,4,self.p) + self.b*pow(z,6,self.p))) % self.p):
                return JacobianPoint(x,y,z,self)
            
    def findPoints(self):
        points = []
        for x in range(self.p):
            for y in range(self.p):
                for z in range(1,self.p):
                    if not((pow(y,2,self.p) - (pow(x,3,self.p) + self.a*x*pow(z,4,self.p) + self.b*pow(z,6,self.p))) % self.p):
                        if JacobianPoint(x,y,z, self) not in points:
                            points.append(JacobianPoint(x,y,z, self))
        points.append(JacobianPoint(1,1,0,self))
        return points

    def isPoint(self,x,y,z):
        return not ((pow(y,2,self.p) - (pow(x,3,self.p) + self.a*x*pow(z,4,self.p) + self.b*pow(z,6,self.p))) % self.p)
    
    def toAffine(self):
        return Curve(self.a,self.b,self.p)

In [14]:
class JacobianPoint:
    def __init__(self,x,y,z, curve, secure = True):
        self.curve = curve
        x %= self.curve.p
        y %= self.curve.p
        z %= self.curve.p
        if not secure:
            if (not self.curve.isPoint(x,y,z)) and z != 0:
                print(x,y,z)
                print("Ошибка! Точка не лежит на кривой!")
                secure = False
            else:
                secure = True
        if secure:
            self.x = x 
            self.y = y 
            self.z = z 
            
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.curve == other.curve) and not ((self.x*other.z**2 - other.x*self.z**2) % self.curve.p) and not ((self.y*other.z**3 - self.z**3*other.y) % self.curve.p)
        else:
            return False
        
    def __repr__(self):
        return f"({self.x}, {self.y}, {self.z})"
    
    def __neg__(self):
        return JacobianPoint(self.x, -self.y, self.z, self.curve)
    
    def doubling(self):
        y1 = self.y
        x1 = self.x
        z1 = self.z
        zz1 = z1*z1 % self.curve.p
        zzz1 = zz1*z1 % self.curve.p
        if y1 == 0:
            return JacobianPoint(1,1,0,self.curve)
        y12 = y1*y1 % self.curve.p
        S = 4*x1*y12  % self.curve.p
        M = (3*x1*x1 % self.curve.p + self.curve.a*zzz1*z1 % self.curve.p)  % self.curve.p
        T = (M*M % self.curve.p - 2*S) % self.curve.p
        x3 = T 
        y3 = M*(S-T) - 8*(y12*y12 % self.curve.p)
        z3 = 2*y1*z1  
        return JacobianPoint(x3,y3,z3, self.curve)
    
    def __add__(self, P):
        y1 = self.y
        x1 = self.x
        z1 = self.z
        y2 = P.y
        x2 = P.x
        z2 = P.z
        if z1 == 0:
            return P
        if z2 == 0:
            return self
        
        zz2 = z2*z2 % self.curve.p
        zz1 = z1*z1 % self.curve.p
        zzz2 = zz2 * z2 % self.curve.p
        zzz1 = zz1*z1 % self.curve.p
        if (self.curve == P.curve) and not ((x1*zz2 - x2*zz1) % self.curve.p) and not ((y1*zzz2 - zzz1*y2) % self.curve.p):
            if y1 == 0:
                return JacobianPoint(1,1,0,self.curve)
            y12 = y1*y1 % self.curve.p
            S = 4*x1*y12  % self.curve.p
            M = (3*x1*x1 % self.curve.p + self.curve.a*zz1*zz1 % self.curve.p)  % self.curve.p
            T = (M*M % self.curve.p - 2*S) % self.curve.p
            x3 = T 
            y3 = M*(S-T) - 8*(y12*y12 % self.curve.p)
            z3 = 2*y1*z1  
            return JacobianPoint(x3,y3,z3, self.curve)
        
        U1 = x1*zz2 % self.curve.p
        U2 = x2*zz1 % self.curve.p
        S1 = y1*zzz2 % self.curve.p
        S2 = y2*zzz1 % self.curve.p
        H = U2 - U1
        H2 = H*H % self.curve.p
        H3 = H2*H % self.curve.p
        r = S2 - S1
        r2 = r*r % self.curve.p
        
        x3 = r2 - H3 - 2*U1*H2
        y3 = r*(U1*H2 % self.curve.p - x3) - S1*H3
        z3 = z1*z2*H 
        return JacobianPoint(x3,y3,z3, self.curve)
        
    def __sub__(self,P):
        return self + (-P)
    
    def toAffine(self):
        curve = self.curve.toAffine()
        if self.z == 0:
            return infPoint(curve)
        return Point(self.x*pow(self.z*self.z % self.curve.p,-1, self.curve.p),self.y*pow(self.z*self.z*self.z % self.curve.p,-1, self.curve.p), curve)
    
    def __mul__(self, m):
        if isinstance(m, int):
            if m < 0:
                m = -m
                self = -self
                
            m = bin(m)[2:]
            result = JacobianPoint(1,1,0,self.curve)
            Q = self
            for i in m[::-1]:
                if i == "1":
                    result = result + Q
                Q += Q
            return result
        else:
            return f"{m} должно быть целым числом"
        
    def __rmul__(self,m):
        return self*m

## Асcоциативность, коммутативность

In [15]:
a = randint(-1000,100)
b = randint(-100,1000)
p = randprime(1<<12, 1<<13)
jc1 = JacobianCurve(a,b,p)
jp1 = jc1.findRandomPoint()
jp2 = jc1.findRandomPoint()
jp3 = jc1.findRandomPoint()
print("Ассоциативность: ", (jp1 + jp2) + jp3 == jp1 + (jp2 + jp3))
print("Коммутативность: ", jp1 + jp2 == jp2 + jp1)

Ассоциативность:  True
Коммутативность:  True


## Другие тесты

In [16]:
a = randint(-1000,100)
b = randint(-100,1000)
p = randprime(1<<12, 1<<13)
jc1 = JacobianCurve(a,b,p)
ec1 = jc1.toAffine()
jp1 = jc1.findRandomPoint()
p1 = jp1.toAffine()
print((p1*b).toJacobian() == jp1*b)
(jp1*a).toAffine() == p1*a # методы перевода

True


True

# Обобщенные координаты Якоби

## Реализация

In [17]:
class ChudnovskyJacobianPoint:            
    def __init__(self,x,y,z,zz,zzz,curve, secure = True):
        self.curve = curve
        x = x % self.curve.p
        y = y % self.curve.p
        z = z % self.curve.p
        zz = zz % self.curve.p
        zzz = zzz % self.curve.p
        if not secure:
            if (not self.curve.isPoint(x,y,z,zz,zzz)) and z != 0:
                print(x,y,z,zz,zzz)
                print("Ошибка! Точка не лежит на кривой!")
                secure = False
            else:
                secure = True
        if secure:
            self.x = x
            self.y = y 
            self.z = z 
            self.zz = zz
            self.zzz = zzz     
            
    def __eq__(self, other): 
        if isinstance(other, self.__class__):
            return (self.curve == other.curve) and not ((self.x*other.zz - other.x*self.zz) % self.curve.p) and not ((self.y*other.zzz - self.zzz*other.y) % self.curve.p)
        else:
            return False
        
    def __repr__(self):
        return f"({self.x}, {self.y}, {self.z}, {self.zz}, {self.zzz})"
    
    def __neg__(self):
        return  ChudnovskyJacobianPoint(self.x, -self.y, self.z, self.zz, self.zzz, self.curve)
    
    def doubling(self):
        y1 = self.y
        x1 = self.x
        z1 = self.z
        zz1 = self.zz
        zzz1 = self.zzz
        if y1 == 0:
                return ChudnovskyJacobianPoint(0,1,0,0,0, self.curve)
        y12 = y1*y1 % self.curve.p
        S = 4*x1*y12  % self.curve.p
        M = (3*(x1*x1 % self.curve.p) + self.curve.a*(zz1*zz1 % self.curve.p)) 
        T = (M*M - 2*S)  % self.curve.p
        x3 = T
        y3 = M*(S-T) - 8*(y12*y12 % self.curve.p)  
        z3 = 2*y1*z1  % self.curve.p
        zz3 = z3*z3 % self.curve.p
        zzz3 = zz3*z3
        return ChudnovskyJacobianPoint(x3,y3,z3,zz3,zzz3, self.curve)
    
    def __add__(self, P):
        y1 = self.y
        x1 = self.x
        z1 = self.z
        zz1 = self.zz
        zzz1 = self.zzz
        y2 = P.y
        x2 = P.x
        z2 = P.z
        zz2 = P.zz
        zzz2 = P.zzz
        
        if z1 == 0:
            return P
        if z2 == 0:
            return self
        if P == self:
            if y1 == 0:
                return ChudnovskyJacobianPoint(0,1,0,0,0, self.curve)
            y12 = y1*y1 % self.curve.p
            S = 4*x1*y12  % self.curve.p
            M = (3*(x1*x1 % self.curve.p) + self.curve.a*(zz1*zz1 % self.curve.p)) 
            T = (M*M - 2*S)  % self.curve.p
            x3 = T
            y3 = M*(S-T) - 8*(y12*y12 % self.curve.p)  
            z3 = 2*y1*z1  % self.curve.p
            zz3 = z3*z3 % self.curve.p
            zzz3 = zz3*z3
            return ChudnovskyJacobianPoint(x3,y3,z3,zz3,zzz3, self.curve)
        
        U1 = x1*zz2 % self.curve.p
        U2 = x2*zz1 % self.curve.p
        S1 = y1*zzz2 % self.curve.p
        S2 = y2*zzz1 % self.curve.p
        H = U2 - U1
        H2 = H*H % self.curve.p
        H3 = H2*H % self.curve.p
        r = S2 - S1
        r2 = r*r % self.curve.p
        
        x3 = r2 - 2*U1*H2 - H3
        y3 = r*(U1*H2 - x3) - S1*H3
        z3 = z1*z2*H % self.curve.p
        zz3 = z3*z3 % self.curve.p
        zzz3 = zz3*z3
        return ChudnovskyJacobianPoint(x3,y3,z3,zz3,zzz3, self.curve)
        
    def __sub__(self,P):
        return self + (-P)
    
    def toAffine(self):
        curve = self.curve.toAffine()
        if self.z == 0:
            return infPoint(curve)
        return Point(self.x*pow(self.zz,-1, self.curve.p),self.y*pow(self.zzz,-1, self.curve.p), curve)
    
    def __mul__(self, m):
        if isinstance(m, int):
            if m < 0:
                m = -m
                self = -self
                
            m = bin(m)[2:]
            result = ChudnovskyJacobianPoint(0,1,0,0,0,self.curve)
            Q = self
            for i in m[::-1]:
                if i == "1":
                    result = result + Q
                Q += Q
            return result
        else:
            return f"{m} должно быть целым числом"
        
    def __rmul__(self,m):
        return self*m

In [18]:
class ChudnovskyJacobianCurve():
    def __init__(self, a, b, p):
        if D(a,b) % p: 
            self.a = a % p
            self.b = b % p
            self.p = p
        else:
            print("Ошибка! Дискриминант кривой нулевой!")
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.a == other.a) and (self.b == other.b) and (self.p == other.p)
        else:
            return False
        
    def findRandomPoint(self):
        while True:
            x = randint(0, self.p - 1)
            y = randint(0, self.p - 1)
            z = randint(1, self.p - 1)
            zz = pow(z,2, self.p)
            zzz = z*zz % self.p
            if not ((pow(y,2,self.p) - (pow(x,3,self.p) + self.a*x*pow(zz,2,self.p) + self.b*pow(zzz,2,self.p))) % self.p):
                return ChudnovskyJacobianPoint(x,y,z, zz, zzz, self)
            
    def findPoints(self):
        points = []
        for x in range(self.p):
            for y in range(self.p):
                for z in range(1,self.p):
                    zz = pow(z,2, self.p)
                    zzz = z*zz % self.p
                    if not ((pow(y,2,self.p) - (pow(x,3,self.p) + self.a*x*pow(zz,2,self.p) + self.b*pow(zzz,2,self.p))) % self.p):
                        if ChudnovskyJacobianPoint(x,y,z,zz,zzz, self) not in points:
                            points.append(ChudnovskyJacobianPoint(x,y,z,zz, zzz, self))
        points.append(ChudnovskyJacobianPoint(1,1,0,0,0,self))
        return points

    def isPoint(self,x,y,z,zz,zzz):
        return not ((pow(y,2,self.p) - (pow(x,3,self.p) + self.a*x*pow(zz,2,self.p) + self.b*pow(zzz,2,self.p))) % self.p)

    def toAffine(self):
        return Curve(self.a,self.b,self.p)

## Асcоциативность, коммутативность

In [19]:
a = randint(-1000,100)
b = randint(-100,1000)
p = randprime(1<<12, 1<<13)
cjc1 = ChudnovskyJacobianCurve(a,b,p)
cjp1 = cjc1.findRandomPoint()
cjp2 = cjc1.findRandomPoint()
cjp3 = cjc1.findRandomPoint()
print("Ассоциативность: ", (cjp1 + cjp2) + cjp3 == cjp1 + (cjp2 + cjp3))
print("Коммутативность: ", cjp1 + cjp2 == cjp2 + cjp1)

Ассоциативность:  True
Коммутативность:  True


## Другие тесты

In [20]:
a = randint(-1000,100)
b = randint(-100,1000)
p = randprime(1<<12, 1<<13)
cjc1 = ChudnovskyJacobianCurve(a,b,p)
ec1 = cjc1.toAffine()
cjp1 = cjc1.findRandomPoint()
p1 = cjp1.toAffine()

print((p1*b).toChudnovskyJacobian() == cjp1*b)
(cjp1*a).toAffine() == p1*a # методы перевода

True


True

# Скорость вычислений

In [38]:
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
a = 0
b = 7
SECP256k1 = Curve(a, b, p)

### Удвоение точки

In [57]:
def doubl(P, tries = 10):
    for _ in range(tries):
        P = P.doubling()
    return P

In [58]:
p1 = SECP256k1.findRandomPoint()
pp1 = p1.toProjective()
jp1 = p1.toJacobian()
cjp1 = p1.toChudnovskyJacobian()

In [59]:
tries = 2**19

start_time = time.time()
doubl(p1, tries)
print("Аффинные координаты:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
doubl(pp1, tries).toAffine()
print("Проективные координаты:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
doubl(jp1, tries).toAffine()
print("Координаты Якоби:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
doubl(cjp1, tries).toAffine()
print("Обобщенные координаты Якоби:--- %s seconds ---" % (time.time() - start_time))

Аффинные координаты:--- 22.50896120071411 seconds ---
Проективные координаты:--- 5.310141324996948 seconds ---
Координаты Якоби:--- 5.354511976242065 seconds ---
Обобщенные координаты Якоби:--- 5.073979139328003 seconds ---


In [25]:
p1 = SECP256k1.findRandomPoint()
pp1 = p1.toProjective()
jp1 = p1.toJacobian()
cjp1 = p1.toChudnovskyJacobian()

In [26]:
tries = 2**20

start_time = time.time()
doubl(pp1, tries).toAffine()
print("Проективные координаты:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
doubl(jp1, tries).toAffine()
print("Координаты Якоби:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
doubl(cjp1, tries).toAffine()
print("Обобщенные координаты Якоби:--- %s seconds ---" % (time.time() - start_time))

Проективные координаты:--- 10.578265905380249 seconds ---
Координаты Якоби:--- 9.88436222076416 seconds ---
Обобщенные координаты Якоби:--- 10.90627670288086 seconds ---


### Сложение (предположительно) различных точек

In [62]:
def addition(P, Q, tries = 10):
    for _ in range(tries):
        P = P + Q
        Q = Q + P

In [63]:
p1, p2 = SECP256k1.findRandomPoint(), SECP256k1.findRandomPoint()
pp1, pp2 = p1.toProjective(), p2.toProjective()
jp1, jp2 = p1.toJacobian(), p2.toJacobian()
cjp1, cjp2 = p1.toChudnovskyJacobian(), p2.toChudnovskyJacobian()

In [64]:
tries = 2**18

start_time = time.time()
addition(p1, p2, tries)
print("Аффинные координаты:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
addition(pp1, pp2,  tries)
print("Проективные координаты:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
addition(jp1, jp2, tries)
print("Координаты Якоби:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
addition(cjp1, cjp2, tries)
print("Обобщенные координаты Якоби:--- %s seconds ---" % (time.time() - start_time))

Аффинные координаты:--- 20.042898654937744 seconds ---
Проективные координаты:--- 10.82426643371582 seconds ---
Координаты Якоби:--- 9.010782241821289 seconds ---
Обобщенные координаты Якоби:--- 8.495838403701782 seconds ---


In [30]:
tries = 2**19

start_time = time.time()
addition(pp1, pp2,  tries)
print("Проективные координаты:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
addition(jp1, jp2, tries)
print("Координаты Якоби:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
addition(cjp1, cjp2, tries)
print("Обобщенные координаты Якоби:--- %s seconds ---" % (time.time() - start_time))

Проективные координаты:--- 22.997071266174316 seconds ---
Координаты Якоби:--- 18.552205085754395 seconds ---
Обобщенные координаты Якоби:--- 16.95832586288452 seconds ---


### Скалярное умножение

In [31]:
a = 2**50-1
bin(a)

'0b11111111111111111111111111111111111111111111111111'

In [32]:
p1 = SECP256k1.findRandomPoint()
pp1 = p1.toProjective()
jp1 = p1.toJacobian()
cjp1 = p1.toChudnovskyJacobian()

In [33]:
s = 10
start_time = time.time()
for _ in range(2**s):
    a*p1
print("Аффинные координаты:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(2**s):
    (a*pp1).toAffine()
print("Проективные координаты:--- %s seconds ---" % (time.time() - start_time))
print(a*p1 == (a*pp1).toAffine())

start_time = time.time()
for _ in range(2**s):
    (a*jp1).toAffine()
print("Координаты Якоби:--- %s seconds ---" % (time.time() - start_time))
print(a*p1 == (a*jp1).toAffine())

start_time = time.time()
for _ in range(2**s):
    (a*cjp1).toAffine()
print("Обобщенные координаты Якоби:--- %s seconds ---" % (time.time() - start_time))
print(a*p1 == (a*cjp1).toAffine())

Аффинные координаты:--- 4.177379608154297 seconds ---
Проективные координаты:--- 2.109037399291992 seconds ---
True
Координаты Якоби:--- 2.109881639480591 seconds ---
True
Обобщенные координаты Якоби:--- 1.4714100360870361 seconds ---
True


In [34]:
p1 = SECP256k1.findRandomPoint()
pp1 = p1.toProjective()
jp1 = p1.toJacobian()
cjp1 = p1.toChudnovskyJacobian()
a = randint(2**40,2**60)
bin(a)

'0b1100000100011111001011111011000101101111111100000010011110'

In [35]:
s = 10
start_time = time.time()
for _ in range(2**s):
    a*p1
print("Аффинные координаты:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(2**s):
    (a*pp1).toAffine()
print("Проективные координаты:--- %s seconds ---" % (time.time() - start_time))
print(a*p1 == (a*pp1).toAffine())

start_time = time.time()
for _ in range(2**s):
    (a*jp1).toAffine()
print("Координаты Якоби:--- %s seconds ---" % (time.time() - start_time))
print(a*p1 == (a*jp1).toAffine())

start_time = time.time()
for _ in range(2**s):
    (a*cjp1).toAffine()
print("Обобщенные координаты Якоби:--- %s seconds ---" % (time.time() - start_time))
print(a*p1 == (a*cjp1).toAffine())

Аффинные координаты:--- 3.729386806488037 seconds ---
Проективные координаты:--- 1.835113763809204 seconds ---
True
Координаты Якоби:--- 1.5577855110168457 seconds ---
True
Обобщенные координаты Якоби:--- 1.3425955772399902 seconds ---
True


In [36]:
p1 = SECP256k1.findRandomPoint()
pp1 = p1.toProjective()
jp1 = p1.toJacobian()
cjp1 = p1.toChudnovskyJacobian()

In [37]:
s = 10
start_time = time.time()
for _ in range(2**s):
    a = randint(2**40,2**50)
    a*p1
print("Аффинные координаты:--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
for _ in range(2**s):
    a = randint(2**40,2**50)
    (a*pp1).toAffine()
print("Проективные координаты:--- %s seconds ---" % (time.time() - start_time))
print(a*p1 == (a*pp1).toAffine())

start_time = time.time()
for _ in range(2**s):
    a = randint(2**40,2**50)
    (a*jp1).toAffine()
print("Координаты Якоби:--- %s seconds ---" % (time.time() - start_time))
print(a*p1 == (a*jp1).toAffine())

start_time = time.time()
for _ in range(2**s):
    a = randint(2**40,2**50)
    (a*cjp1).toAffine()
print("Обобщенные координаты Якоби:--- %s seconds ---" % (time.time() - start_time))
print(a*p1 == (a*cjp1).toAffine())

Аффинные координаты:--- 3.001615524291992 seconds ---
Проективные координаты:--- 1.9091746807098389 seconds ---
True
Координаты Якоби:--- 1.297048807144165 seconds ---
True
Обобщенные координаты Якоби:--- 1.173142433166504 seconds ---
True
